- Update: 2023.02.04

# 0. 配置参数

- `prepared_alternate`：整理好的替代料表路径
- `price`：改价表的相关信息
    - `file_name`：文件的路径
    - `sheet_name`：工作表名称
    - `col_name`：SKU号所在列的列名
- `mrp`：MRP表路径
- `out`：保存结果的路径

In [1]:
prepared_alternate = '../../file/Lennon_断货汇报_整理好的替代料表_20230204_085144.xlsx'
dict_price = {'file_name': '../../file/改价表 2-3.xlsx',
              'sheet_name': 'YDK2库存 02-03',
              'col_name': '商品编码 2-3'}
mrp = '../../file/US MRP20230130运营.xlsx'

out = '../../file/Lennon_断货汇报_整理好的库存表_NOW().xlsx'

In [2]:
import pandas as pd

In [3]:
df_prepared_alternate = pd.read_excel(prepared_alternate, header=0).fillna('')

df_prepared_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,
1,USBDA04108,USBDA04108X-C,,,,,
2,USGS43640B,USGS43640,,,,,
3,USWS02863,,,,,,
4,USWS02632,,,,,,
...,...,...,...,...,...,...,...
49913,USCAK20801,,,,,,
49914,USWR52641MU,USWR52641M,,,,,
49915,USWR832FRJHY,USWR832FRJ,USWR41832,USWR41832J,,,
49916,USDIF41300-C,,,,,,


In [4]:
df_price = pd.read_excel(dict_price['file_name'], sheet_name=dict_price['sheet_name'], header=0)[[dict_price['col_name'], '可售']]
df_price = df_price[df_price['可售'] != 0].reset_index(drop=True)
df_price = df_price.groupby([dict_price['col_name']], as_index=False).sum()

df_price

,商品编码 2-3,可售
0,USAAS64500,1
1,USAAS64500-N,1
2,USABS00021,8
3,USABS00021-N,1
4,USABS00117,7
...,...,...
52656,USWT90120,3
52657,USWT90220,8
52658,USWT90420,8
52659,USWT90720,10


In [5]:
df_mrp = pd.read_excel(mrp, header=1)[['MRP SKU']]

df_mrp

,MRP SKU
0,USGS06060Z
1,USTCV17215HIE-2-C
2,USTBC40007-C
3,USGS41420B
4,USGSM3570T
...,...
49913,USFM0FD006B0-YC
49914,USASV98007
49915,USASC3B484
49916,USCAK53670


# 1. 计算替代料表库存

In [6]:
df_prepared_alternate = pd.merge(left=df_prepared_alternate, right=df_price, how='left', left_on='主料SKU', right_on=dict_price['col_name']).fillna(0)
df_prepared_alternate = pd.merge(left=df_prepared_alternate, right=df_price, how='left', left_on='替换料1', right_on=dict_price['col_name'], suffixes=('', '_1')).fillna(0)
df_prepared_alternate = pd.merge(left=df_prepared_alternate, right=df_price, how='left', left_on='替换料2', right_on=dict_price['col_name'], suffixes=('', '_2')).fillna(0)
df_prepared_alternate = pd.merge(left=df_prepared_alternate, right=df_price, how='left', left_on='替换料3', right_on=dict_price['col_name'], suffixes=('', '_3')).fillna(0)
df_prepared_alternate = pd.merge(left=df_prepared_alternate, right=df_price, how='left', left_on='替换料4', right_on=dict_price['col_name'], suffixes=('', '_4')).fillna(0)
df_prepared_alternate = pd.merge(left=df_prepared_alternate, right=df_price, how='left', left_on='替换料5', right_on=dict_price['col_name'], suffixes=('', '_5')).fillna(0)
df_prepared_alternate = pd.merge(left=df_prepared_alternate, right=df_price, how='left', left_on='替换料6', right_on=dict_price['col_name'], suffixes=('', '_6')).fillna(0)
df_prepared_alternate['库存'] = df_prepared_alternate['可售'] + df_prepared_alternate['可售_1'] + df_prepared_alternate['可售_2'] + df_prepared_alternate['可售_3'] + df_prepared_alternate['可售_4'] + df_prepared_alternate['可售_5'] + df_prepared_alternate['可售_6']

df_prepared_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,商品编码 2-3,可售,商品编码 2-3_1,...,可售_2,商品编码 2-3_3,可售_3,商品编码 2-3_4,可售_4,商品编码 2-3_5,可售_5,商品编码 2-3_6,可售_6,库存
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,,USCA171876D-3,10.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,10.0
1,USBDA04108,USBDA04108X-C,,,,,,USBDA04108,291.0,USBDA04108X-C,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,423.0
2,USGS43640B,USGS43640,,,,,,USGS43640B,187.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,187.0
3,USWS02863,,,,,,,USWS02863,222.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,222.0
4,USWS02632,,,,,,,USWS02632,293.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,293.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49913,USCAK20801,,,,,,,USCAK20801,5.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,5.0
49914,USWR52641MU,USWR52641M,,,,,,0,0.0,USWR52641M,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,12.0
49915,USWR832FRJHY,USWR832FRJ,USWR41832,USWR41832J,,,,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0
49916,USDIF41300-C,,,,,,,USDIF41300-C,132.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,132.0


# 2. 计算MRP表库存

In [7]:
df_mrp = pd.merge(left=df_mrp, right=df_prepared_alternate, how='left', left_on='MRP SKU', right_on='主料SKU')
df_mrp = df_mrp[['MRP SKU', '库存']]

df_mrp

,MRP SKU,库存
0,USGS06060Z,296.0
1,USTCV17215HIE-2-C,85.0
2,USTBC40007-C,525.0
3,USGS41420B,56.0
4,USGSM3570T,118.0
...,...,...
49913,USFM0FD006B0-YC,0.0
49914,USASV98007,0.0
49915,USASC3B484,8.0
49916,USCAK53670,10.0


# 3. 保存

In [8]:
import datetime

writer = pd.ExcelWriter(out.replace('NOW()', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")))
df_mrp.to_excel(writer, index=False)
writer.close()